# Account


In [32]:
# | default_exp classes.DomoAccount


In [33]:
# | exporti

from dataclasses import dataclass, field

import datetime as dt
import re

import httpx

from fastcore.basics import patch_to

import domolibrary.utils.convert as cd
import domolibrary.utils.DictDot as util_dd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de
import domolibrary.routes.account as account_routes

import domolibrary.utils.chunk_execution as ce

In [34]:
#| export 

from domolibrary.routes.account import (
    ShareAccount_V1_AccessLevel,
    ShareAccount_V2_AccessLevel,
    ShareAccount,
    GetAccount_NoMatch,
    ShareAccount_Error,
    DeleteAccount_Error
)

from domolibrary.classes.DomoAccount_Config import (
    AccountConfig_UsesOauth,
    AccountConfig_ProviderTypeNotDefined,
    DomoAccount_Config,
    AccountConfig,
)

class Account_CanIModify(de.DomoError):
    def __init__(self, account_id, domo_instance):
        super().__init__(
            message = f"`DomoAccount.is_admin_summary` must be `False` to proceed.  Either set the value explicity, or retrieve the account instance using `DomoAccount.get_by_id()`",
            domo_instance = domo_instance
        )

        
class UpsertAccount_MatchCriteria(de.DomoError):
    def __init__(self, domo_instance):
        super().__init__(
            message="must pass an account_id or account_name to UPSERT",
            domo_instance=domo_instance,
        )



# DomoAccount


In [35]:
# | export
@dataclass
class DomoAccount:
    data_provider_type: str = None
    name: str = None

    id: int = None
    created_dt: dt.datetime = None
    modified_dt: dt.datetime = None
    auth: dmda.DomoAuth = field(repr=False, default=None)

    config: DomoAccount_Config = None,

    is_admin_summary: bool = True

    @classmethod
    def _from_json(cls, obj: dict,
                   is_admin_summary: bool = True,
                   auth: dmda.DomoAuth = None,
                   ):
        """converts data_v1_accounts API response into an accounts class object"""

        dd = util_dd.DictDot(obj)

        return cls(
            id=dd.id or dd.databaseId,
            name=dd.displayName,
            data_provider_type=dd.dataProviderId or dd.dataProviderType,
            created_dt=cd.convert_epoch_millisecond_to_datetime(
                dd.createdAt or dd.createDate
            ),
            modified_dt=cd.convert_epoch_millisecond_to_datetime(
                dd.modifiedAt or dd.lastModified
            ),
            auth=auth,
            is_admin_summary=is_admin_summary
        )


## DomoAccount GET methods


In [36]:
# | export


@patch_to(DomoAccount)
async def _get_config(
    self: DomoAccount,
    session=None,
    return_raw: bool = False,
    debug_api: bool = None,
    auth: dmda.DomoAuth = None,
    debug_num_stacks_to_drop=2,
    is_suppress_no_config: bool = False,  # can be used to suppress cases where the config is not defined, either because the account_config is OAuth, and therefore not stored in Domo OR because the AccountConfig class doesn't cover the data_type
):
    res = await account_routes.get_account_config(
        auth=auth or self.auth,
        account_id=self.id,
        session=session,
        debug_api=debug_api,
        parent_class=self.__class__.__name__,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if return_raw:
        return res

    data_provider_type = res.response.get("_search_metadata").get("data_provider_type")

    config_fn = AccountConfig(data_provider_type).value

    if not is_suppress_no_config and not config_fn.is_defined_config:
        raise config_fn._associated_exception(data_provider_type)

    self.config = config_fn._from_json(res.response)

    return self.config

##### sample of hidden function \_get_config

called by `DomoAccount.get_by_id()`


In [37]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_account = DomoAccount(
    auth=token_auth,
    id=45,
)

try:
    await domo_account._get_config(
        debug_api=False, return_raw=False, is_suppress_no_config=False
    )

except AccountConfig_UsesOauth as e:
    print(e)

data provider type google-spreadsheets uses OAuth and therefore wouldn't return a Config object
data provider type google-spreadsheets uses OAuth and therefore wouldn't return a Config object


In [38]:
# | export
@patch_to(DomoAccount, cls_method=True)
async def get_by_id(
    cls,
    auth: dmda.DomoAuth,
    account_id: int,
    is_suppress_no_config: bool = True,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
):
    """retrieves account metadata and attempts to retrieve config"""

    res = await account_routes.get_account_from_id(
        auth=auth,
        account_id=account_id,
        session=session,
        debug_api=debug_api,
        parent_class=cls.__name__,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if return_raw:
        return res

    acc = cls._from_json(obj = res.response, auth = auth, is_admin_summary = False)

    await acc._get_config(
        session=session,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop + 1,
        is_suppress_no_config=is_suppress_no_config
    )

    return acc


#### sample implementation of get_by_id

this implementation returns a result because accoun_id 5 is linked to a data_provider (domo_governance) that has been defined in the `DomoAccount_Config.AccountConfig` enum


In [39]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await DomoAccount.get_by_id(
    auth=token_auth, account_id=45, return_raw=False, debug_api=False,
    is_suppress_no_config = True
)

data provider type google-spreadsheets uses OAuth and therefore wouldn't return a Config object


DomoAccount(data_provider_type='google-spreadsheets', name='test-goolesheets', id=45, created_dt=datetime.datetime(2022, 10, 4, 22, 59, 14), modified_dt=datetime.datetime(2023, 4, 24, 15, 40, 28), config=DomoAccount_NoConfig_OAuth(is_oauth=True, is_defined_config=False), is_admin_summary=False)

This implementation fails because the data_provider_type has not been defined in `DomoAccount_Config.AccountConfig`


In [40]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


await DomoAccount.get_by_id(
    auth=token_auth, account_id=45, return_raw=False, debug_api=False
)


data provider type google-spreadsheets uses OAuth and therefore wouldn't return a Config object


DomoAccount(data_provider_type='google-spreadsheets', name='test-goolesheets', id=45, created_dt=datetime.datetime(2022, 10, 4, 22, 59, 14), modified_dt=datetime.datetime(2023, 4, 24, 15, 40, 28), config=DomoAccount_NoConfig_OAuth(is_oauth=True, is_defined_config=False), is_admin_summary=False)

## DomoAccount CRUD Methods


### Create Account


In [41]:
# | export

@staticmethod
@patch_to(DomoAccount)
def generate_create_body(account_name, config):
    return {
        "displayName": account_name,
        "dataProviderType": config.data_provider_type,
        "name": config.data_provider_type,
        "configurations": config.to_json(),
    }


@patch_to(DomoAccount, cls_method=True)
async def create_account(
    cls: DomoAccount,
    account_name: str,
    config: DomoAccount_Config,
    auth: dmda.DomoAuth,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):
    body = cls.generate_create_body(account_name=account_name, config=config)

    res = await account_routes.create_account(
        auth=auth, config_body=body, debug_api=debug_api, session=session
    )


    return await cls.get_by_id(auth=auth, account_id=res.response.get("id"))

### Update DomoAccount


In [42]:
# | export


@patch_to(DomoAccount)
async def update_config(
    self: DomoAccount,
    auth: dmda.DomoAuth = None,
    debug_api: bool = False,
    config: DomoAccount_Config = None,
    is_suppress_no_config=False,
    debug_num_stacks_to_drop = 2,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
):
    if self.is_admin_summary:
        raise Account_CanIModify(account_id= self.id, domo_instance= auth.domo_instance)
        
    auth = auth or self.auth
    config = config or self.config

    res = await account_routes.update_account_config(
        auth=auth,
        account_id=self.id,
        config_body=config.to_json(),
        debug_api=debug_api,
        session=session,
    )

    if return_raw:
        return res

    await self._get_config(
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop + 1,
        is_suppress_no_config=is_suppress_no_config)

    return self


#### Sample implementation of update_config

to update the account config,

- assign `DomoAccount` class instance `domo_account.config` a `DomoAccount` class.<br>
  - the `AccountConfig` enumerates all of the `DomoAccount`s currently available. _issue a pull request to add a new one_
  - each member of `AccountConfig` uses the camel_case spelling of the data-provider-type


In [43]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

# creates a DomoAccount object
domo_account = await DomoAccount.get_by_id(auth=token_auth, account_id=5)


# update domo Account API without passing explicit config object
# adjust configuration information for that object
domo_account.config.domo_instance=  "any_new_value_goes_here"
await domo_account.update_config()

DomoAccount(data_provider_type='domo-governance-d14c2fef-49a8-4898-8ddd-f64998005600', name='test_rename', id=5, created_dt=datetime.datetime(2021, 3, 26, 16, 54, 41), modified_dt=datetime.datetime(2023, 10, 24, 1, 27, 46), config=DomoAccount_Config_Governance(domo_instance='any_new_value_goes_here'), is_admin_summary=False)

The config can be updated by passing a new `AccountConfig` object

In [44]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_account = await DomoAccount.get_by_id(auth=token_auth, account_id=5)

# update domo Account API by passing new config object
config = AccountConfig.domo_governance_d14c2fef_49a8_4898_8ddd_f64998005600.value(
    domo_instance=token_auth.domo_instance, 
    access_token=token_auth.domo_access_token
)
await domo_account.update_config(config=config)

DomoAccount(data_provider_type='domo-governance-d14c2fef-49a8-4898-8ddd-f64998005600', name='test_rename', id=5, created_dt=datetime.datetime(2021, 3, 26, 16, 54, 41), modified_dt=datetime.datetime(2023, 10, 24, 1, 27, 46), config=DomoAccount_Config_Governance(domo_instance='domo-community'), is_admin_summary=False)

In [45]:
# | export
@patch_to(DomoAccount)
async def update_name(
    self: DomoAccount,
    account_name: str = None,
    auth: dmda.DomoAuth = None,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
):
    if self.is_admin_summary:
        raise Account_CanIModify(account_id= self.id, domo_instance= auth.domo_instance)

    auth = auth or self.auth

    res = await account_routes.update_account_name(
        auth=auth,
        account_id=self.id,
        account_name=account_name or self.name,
        debug_api=debug_api,
        session=session,
    )

    if return_raw:
        return res

    await self.get_by_id(auth=auth, account_id=self.id)

    return self

#### sample implementation of update_name

`DomoAccount` can be updated by altering properties and then calling `DomoAccount.update_name()` or by passing the account_name parameter 

In [46]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_account = await DomoAccount.get_by_id(auth=token_auth, account_id=5)
domo_account.name = "abc"
await domo_account.update_name()

DomoAccount(data_provider_type='domo-governance-d14c2fef-49a8-4898-8ddd-f64998005600', name='abc', id=5, created_dt=datetime.datetime(2021, 3, 26, 16, 54, 41), modified_dt=datetime.datetime(2023, 10, 24, 1, 27, 46), config=DomoAccount_Config_Governance(domo_instance='domo-community'), is_admin_summary=False)

In [47]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_account = await DomoAccount.get_by_id(auth=token_auth, account_id=5)

await domo_account.update_name(account_name='test_rename', return_raw=False)

DomoAccount(data_provider_type='domo-governance-d14c2fef-49a8-4898-8ddd-f64998005600', name='abc', id=5, created_dt=datetime.datetime(2021, 3, 26, 16, 54, 41), modified_dt=datetime.datetime(2023, 10, 24, 1, 30, 24), config=DomoAccount_Config_Governance(domo_instance='domo-community'), is_admin_summary=False)

### Delete Account


In [48]:
# | export


@patch_to(DomoAccount)
async def delete_account(
    self: DomoAccount,
    auth: dmda.DomoAuth = None,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    debug_num_stacks_to_drop=2,
):
    if self.is_admin_summary:
        raise Account_CanIModify(account_id= self.id, domo_instance= auth.domo_instance)

    auth = auth or self.auth

    res = await account_routes.delete_account(
        auth=auth,
        account_id=self.id,
        debug_api=debug_api,
        session=session,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
    )

    return res


### Share DomoAccount


In [49]:
# | export


@patch_to(DomoAccount)
async def _is_group_ownership_beta(self, auth: dmda.DomoAuth):
    import domolibrary.classes.DomoBootstrap as dmbs

    domo_bsr = dmbs.DomoBootstrap(auth=auth or self.auth)

    domo_feature_ls = await domo_bsr.get_features()

    match_accounts_v2 = next(
        (
            domo_feature
            for domo_feature in domo_feature_ls
            if domo_feature.name == "accounts-v2"
        ),
        None,
    )

    return True if match_accounts_v2 else False


@patch_to(DomoAccount)
async def share_account(
    self,
    user_id: int = None,
    group_id: int = None,
    domo_user=None,  # DomoUser,
    domo_group=None,  # DomoGroup
    auth: dmda.DomoAuth = None,
    is_v2: bool = None,
    access_level: ShareAccount = None,  # will default to Read
    debug_api: bool = False,
    debug_prn: bool = False,
    session: httpx.AsyncClient = None,
):
    if self.is_admin_summary:
        raise Account_CanIModify(account_id= self.id, domo_instance= auth.domo_instance)

    auth = auth or self.auth

    user_id = user_id or (domo_user and domo_user.id)
    group_id = group_id or (domo_group and domo_group.id)

    if isinstance(auth, dmda.DomoFullAuth) and is_v2 is None:
        is_v2 = await self._is_group_ownership_beta(auth)

    if debug_prn:
        print(
            f"ℹ️ - {auth.domo_instance} - {'is' if is_v2 else 'is not'} v2_group_ownership"
        )

    if is_v2 is None:
        raise Exception(
            """🛑 ERROR must explicitly pass a value for the `is_v2` boolean to share_accounts function.ABC
alternatively, use `dmda.DomoFullAuth` to automatically retrieve the correct setting.ABC
account sharing differs between v1 and v2 of the API."""
        )

    res = None

    if is_v2:
        share_payload = account_routes.generate_share_account_payload_v2(
            user_id=user_id,
            group_id=group_id,
            access_level=access_level or ShareAccount_V2_AccessLevel.CAN_VIEW,
        )

        res = await account_routes.share_account_v2(
            auth=auth,
            account_id=self.id,
            share_payload=share_payload,
            debug_api=debug_api,
            session=session,
        )

    else:
        share_payload = account_routes.generate_share_account_payload_v1(
            user_id=user_id,
            group_id=group_id,
            access_level=access_level or ShareAccount_V1_AccessLevel.CAN_VIEW,
        )

        res = await account_routes.share_account_v1(
            auth=auth,
            account_id=self.id,
            share_payload=share_payload,
            debug_api=debug_api,
            session=session,
        )

    if not res.is_success:
        return None

    if res.status == 200:
        res.response = f"shared {self.id} - {self.name} with {user_id or group_id}"

    return res


#### sample implementation of share_account

Below, we are using `DomoAuth` to hit the `DomoAccount.share_account`.

To do this, we must first know whether we are hitting a v2 or v1 sharing API because they require different API bodies.

If you use `DomoFullAuth` the class can hit the bootstrap API to test whether it's V1 or V2.
If you use `TokenAuth` you must specify which method to attempt (becuase TokenAuth cannot hit the Bootstrap API)


In [50]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_account = await DomoAccount.get_by_id(account_id=5, auth=token_auth)

res = None
try:
    print("demonstrate needing to pass is_v2 if using token auth")
    res = await domo_account.share_account(user_id=1893952720)
except Exception as e:
    print(e)

res

demonstrate needing to pass is_v2 if using token auth
🛑 ERROR must explicitly pass a value for the `is_v2` boolean to share_accounts function.ABC
alternatively, use `dmda.DomoFullAuth` to automatically retrieve the correct setting.ABC
account sharing differs between v1 and v2 of the API.


In [51]:
# | exporti


@patch_to(DomoAccount)
async def share(
    self: DomoAccount,
    domo_user=None,
    domo_group=None,
    auth: dmda.DomoAuth = None,
    is_v2: bool = None,
    access_level: ShareAccount = None,  # will default to Read
    debug_api: bool = False,
    debug_num_stacks_to_drop: int = 2,
    debug_prn: bool = False,
    session: httpx.AsyncClient = None,
):
    if self.is_admin_summary:
        raise Account_CanIModify(account_id= self.id, domo_instance= auth.domo_instance)

    auth = auth or self.auth

    if isinstance(auth, dmda.DomoFullAuth) and is_v2 is None:
        is_v2 = await self._is_group_ownership_beta(auth)

    if debug_prn:
        print(
            f"ℹ️ - {auth.domo_instance} - {'is' if is_v2 else 'is not'} v2_group_ownership"
        )

    if is_v2 is None:
        raise Exception(
            """🛑 ERROR must pass `is_v2` bool to share_accounts function IF NOT passing `dmda.DomoFullAuth`.
the group management v2 API has a different body.  
Alternatively pass a full auth object to auto check the bootstrap.
"""
        )

    res = None

    if is_v2:
        share_payload = account_routes.generate_share_account_payload_v2(
            user_id=domo_user.id if domo_user else None,
            group_id=domo_group.id if domo_group else None,
            access_level=access_level or ShareAccount_V2_AccessLevel.CAN_VIEW,
        )

        res = await account_routes.share_account_v2(
            auth=auth,
            account_id=self.id,
            share_payload=share_payload,
            debug_api=debug_api,
            debug_num_stacks_to_drop=debug_num_stacks_to_drop,
            parent_class=self.__class__.__name__,
            session=session,
        )

    else:
        share_payload = account_routes.generate_share_account_payload_v1(
            user_id=domo_user.id if domo_user else None,
            group_id=domo_group.id if domo_group else None,
            access_level=access_level or ShareAccount_V1_AccessLevel.CAN_VIEW,
        )

        res = await account_routes.share_account_v1(
            auth=auth,
            account_id=self.id,
            share_payload=share_payload,
            debug_api=debug_api,
            session=session,
            debug_num_stacks_to_drop=debug_num_stacks_to_drop,
            parent_class=self.__class__.__name__,
        )

    if not res.is_success:
        return None

    if res.status == 200:
        domo_entity = domo_user or domo_group
        res.response = f"shared {self.id} - {self.name} with {domo_entity.id}"

    return res


#### sample implementation of domo_account.share


In [52]:
import os
import domolibrary.client.DomoAuth as dmda
import domolibrary.classes.DomoGroup as dmg

full_auth = dmda.DomoFullAuth(
    domo_instance="domo-community",
    domo_username=os.environ["DOMO_USERNAME"],
    domo_password=os.environ["DOJO_PASSWORD"],
)
account_id = 5
domo_account = await DomoAccount.get_by_id(account_id=account_id, auth=full_auth)

domo_group = await dmg.DomoGroup.get_by_id(group_id=1814479647, auth=token_auth)

try:
    await domo_account.share(
        domo_group=domo_group, auth=full_auth, debug_prn=False, debug_api=False
    )

except ShareAccount_Error as e:
    print(e)


🛑  ShareAccount_Error 🛑 - functionn: DomoAccount.share || 5 || status 500 || ℹ️ - Internal Server Error| User may already have access to account. at domo-community


# DomoAccounts

A class for retrieving account objects


In [53]:
# | export


@dataclass
class DomoAccounts:
    auth: dmda.DomoAuth


In [54]:
# | export

@staticmethod
@patch_to(DomoAccounts)
async def _get_accounts_accountsapi(
    auth: dmda.DomoAuth,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
):
    res = await account_routes.get_accounts(
        auth=auth, debug_api=debug_api, session=session
    )

    if return_raw:
        return res
    
    if  len(res.response) == 0: 
        return []

    return await ce.gather_with_concurrency(
        n=60,
        *[
            DomoAccount.get_by_id(
                account_id=json_obj.get("id"),
                debug_api=debug_api,
                session=session,
                auth=auth,
            )
            for json_obj in res.response
        ],
    )


@patch_to(DomoAccounts, cls_method=True)
async def _get_accounts_queryapi(
    cls: DomoAccounts,
    auth: dmda.DomoAuth,
    debug_api: bool = False,
    additional_filters_ls=None,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
):
    """v2 api for works with group_account_v2 beta"""

    import domolibrary.routes.datacenter as datacenter_routes

    res = await datacenter_routes.search_datacenter(
        auth=auth,
        entity_type=datacenter_routes.Datacenter_Enum.ACCOUNT.value,
        additional_filters_ls=additional_filters_ls,
        session=session,
        debug_api=debug_api,
    )

    if return_raw:
        return res
    
    if  len(res.response) == 0:
        return []
        
    return [
        DomoAccount._from_json(account_obj, auth=auth) for account_obj in res.response
    ]


@patch_to(DomoAccounts, cls_method=True)
async def get_accounts(
    cls: DomoAccounts,
    auth: dmda.DomoAuth,
    additional_filters_ls=None,  # datacenter_routes.generate_search_datacenter_filter
    # account string to search for, must be an exact match in spelling.  case insensitive
    # v2 will use the queryAPI as it returns more complete results than the accountsAPI
    is_v2: bool = None,
    is_suppress_undefined_provider_type: bool = False,
    account_name: str = None,
    account_id: str = None,
    account_type: AccountConfig = None,  # to retrieve a specific account type
    account_type_str=None,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    return_raw: bool = False,
    debug_prn: bool = False,
):
    import domolibrary.classes.DomoBootstrap as bsr
    import domolibrary.routes.datacenter as datacenter_routes

    if isinstance(auth, dmda.DomoFullAuth) and is_v2 is None:
        instance_bsr = bsr.DomoBootstrap(auth=auth)

        is_v2 = await instance_bsr.is_group_ownership_beta(auth)

        if debug_prn:
            print(
                f"{auth.domo_instance} {'is' if is_v2 else 'is not'} using the v2 beta"
            )

    if is_v2:
        try:
            domo_accounts = await cls._get_accounts_queryapi(
                auth=auth,
                debug_api=debug_api,
                additional_filters_ls=additional_filters_ls,
                session=session,
            )
        except datacenter_routes.SearchDatacenter_NoResultsFound as e:
            print(e)
            domo_accounts = []
    else:
        domo_accounts = await cls._get_accounts_accountsapi(
            auth=auth, debug_api=debug_api, session=session
        )

    if return_raw or len(domo_accounts) == 0:
        return domo_accounts

    if account_id:
        domo_account = next(
            (
                domo_account
                for domo_account in domo_accounts
                if int(domo_account.id) == int(account_id)
            ),
            None,
        )

        if not domo_account:
            raise GetAccount_NoMatch(
                account_id=account_id, domo_instance=auth.domo_instance
            )

        return domo_account

    if account_name and isinstance(account_name, str):
        domo_accounts = [
            domo_account
            for domo_account in domo_accounts
            if domo_account.name.lower() == account_name.lower()
        ]

    if account_type:
        return [
            domo_account
            for domo_account in domo_accounts
            if domo_account.data_provider_type == account_type.value.data_provider_type
        ]

    if account_type_str:
        return [
            domo_account
            for domo_account in domo_accounts
            if domo_account.data_provider_type == account_type_str
        ]

    return domo_accounts


#### sample get_accounts


In [55]:
import os

auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await DomoAccounts.get_accounts(
    auth=auth,
    is_v2=True,
    # account_name = 'domo_creds',
    account_id=73,
    return_raw=False,
    debug_prn=False,
    debug_api=False,
)

res


DomoAccount(data_provider_type='google-bigquery-service', name='Google BigQuery Service Account', id='73', created_dt=datetime.datetime(2023, 6, 26, 4, 18, 51), modified_dt=datetime.datetime(2023, 6, 26, 4, 18, 51), config=(None,), is_admin_summary=True)

In [56]:
# | exporti
@patch_to(DomoAccount)
async def get_accesslist(
    self: DomoAccount,
    auth: dmda.DomoAuth = None,
    debug_api: bool = False,
    return_raw: bool = False,
    session: httpx.AsyncClient = None,
):
    res = await account_routes.get_account_accesslist_for_v2(
        auth=auth or self.auth, account_id=self.id, debug_api=debug_api, session=session
    )

    if return_raw:
        return res

    return res.response["list"]


#### sample implementation of get_accesslist


In [57]:
import os

auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

# creates a DomoAccount object
domo_account = await DomoAccount.get_by_id(auth=auth, account_id=5)

await domo_account.get_accesslist()

[{'type': 'USER',
  'id': '587894148',
  'accessLevel': 'CAN_VIEW',
  'name': 'Bryan Van Kampen'},
 {'type': 'USER',
  'id': '612085674',
  'accessLevel': 'CAN_VIEW',
  'name': 'Oleksii Zakrevskyi'},
 {'type': 'USER',
  'id': '1893952720',
  'accessLevel': 'OWNER',
  'name': 'Jae Wilson1'},
 {'type': 'USER',
  'id': '68216396',
  'accessLevel': 'CAN_VIEW',
  'name': 'Elliott Leonard'},
 {'type': 'USER', 'id': '1256759792', 'accessLevel': 'CAN_VIEW'}]

# UPSERT Functions


In [58]:
# | exporti

@patch_to(DomoAccounts, cls_method=True)
async def upsert_account(
    cls: DomoAccounts,
    auth: dmda.DomoAuth,

    account_config: AccountConfig = None, 
    account_name: str = None,
    account_id: str = None,
    debug_api: bool = False,
    debug_prn: bool = False,
    session: httpx.AsyncClient = None
):
    """search for an account and upsert it"""

    if not account_name and not account_id:
        raise UpsertAccount_MatchCriteria(domo_instance=auth.domo_instance)

    acc = None

    if account_id:
        acc = await DomoAccounts.get_accounts(account_id=account_id, auth=auth)

        if acc and account_name:
            if debug_prn:
                print(f"upsertting {acc.id}:  updating account_name")
            await acc.update_name(account_name=account_name, debug_api = debug_api)

    if account_name and acc is None:
        acc = await DomoAccounts.get_accounts(
            account_name=account_name, auth=auth,
            account_type_str= (account_config and account_config.data_provider_type) or None,
            # is_suppress_undefined_provider_type = True
        )

        if (
            isinstance(acc, list)
            and len(acc) > 0
            and isinstance(acc[0], DomoAccount)
        ):
            acc = acc[0]

        else:
            acc = None


    if acc and account_config:  # upsert account
        acc.config = account_config

        if debug_prn:
            print(f"upsertting {acc.id}:  updating config")

        await acc.update_config( debug_api = debug_api)

    if not acc:
        if debug_prn:
            print(f"creating account {account_name} in {auth.domo_instance}")
            
        acc = await DomoAccount.create_account(
            account_name=account_name,
            config=account_config,
            auth=auth,
            debug_api=debug_api,
        )

    return acc


#### sample implementation of upsert_account


In [59]:
import os
import domolibrary.classes.DomoUser as dmu
import datetime as dt

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)
await DomoAccount.get_by_id(
    account_id=71,
    auth=token_auth,
)


DomoAccount(data_provider_type='abstract-credential-store', name='domolibrary test account - updated 2023-10-24', id=71, created_dt=datetime.datetime(2023, 5, 18, 21, 58, 12), modified_dt=datetime.datetime(2023, 10, 24, 1, 30, 16), config=DomoAccount_Config_AbstractCredential(credentials='********'), is_admin_summary=False)

In [60]:
import os
import domolibrary.classes.DomoUser as dmu
import datetime as dt

dt.date.today()

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

account_config = AccountConfig.abstract_credential_store.value(
    credentials=f"abc {dt.date.today()}"
)


# creates a DomoAccount object
await DomoAccounts.upsert_account(
    account_name=f"domo_creds - update {dt.date.today()}",
    account_id=71,
    auth=token_auth,
    debug_api=False,
    debug_prn=True,
    account_config=account_config,
)


data provider type domo-csv not defined yet. Extend the AccountConfig class
data provider type google-spreadsheets uses OAuth and therefore wouldn't return a Config object
data provider type google-spreadsheets uses OAuth and therefore wouldn't return a Config object
upsertting 71:  updating account_name
upsertting 71:  updating config


DomoAccount(data_provider_type='abstract-credential-store', name='domolibrary test account - updated 2023-10-24', id=71, created_dt=datetime.datetime(2023, 5, 18, 21, 58, 12), modified_dt=datetime.datetime(2023, 10, 24, 1, 30, 16), config=DomoAccount_Config_AbstractCredential(credentials='********'), is_admin_summary=False)

In [63]:
# | exporti


@patch_to(DomoAccount)
async def upsert_share_account_user(
    self: DomoAccount,
    domo_user,
    auth: dmda.DomoAuth = None,
    is_v2: bool = None,
    access_level: ShareAccount = None,  # will default to Read
    debug_api: bool = False,
    debug_prn: bool = False,
    session: httpx.AsyncClient = None,
):
    auth = auth or self.auth

    ls_share = await account_routes.get_account_accesslist_for_v2(auth=auth, account_id=self.id)
    res = None
    if domo_user:
        user_id = domo_user.id
        found_user = next(
            (
                obj
                for obj in ls_share.response["list"]
                if obj["id"] == user_id and obj["type"] == "USER"
            ),
            None,
        )
        if not found_user:
            res = await self.share(
                domo_user=domo_user,
                auth=auth,
                is_v2=is_v2,
                access_level=access_level,
                debug_api=debug_api,
                debug_prn=debug_prn,
                session=session,
            )

    return res


@patch_to(DomoAccount)
async def upsert_share_account_group(
    self: DomoAccount,
    domo_group,
    auth: dmda.DomoAuth = None,
    is_v2: bool = None,
    access_level: ShareAccount = None,  # will default to Read
    debug_api: bool = False,
    debug_prn: bool = False,
    session: httpx.AsyncClient = None,
):
    auth = auth or self.auth

    ls_share = await account_routes.get_account_accesslist_for_v2(auth=auth, account_id=self.id)
    res = None

    if domo_group:
        group_id = domo_group.id
        found_group = next(
            (
                obj
                for obj in ls_share.response["list"]
                if obj["id"] == group_id and obj["type"] == "GROUP"
            ),
            None,
        )
        if not found_group:
            res = await self.share(
                domo_group=domo_group,
                auth=auth,
                is_v2=is_v2,
                access_level=access_level,
                debug_api=debug_api,
                debug_prn=debug_prn,
                session=session,
            )

    return res


#### sample implementation upsert share account


In [64]:
import os
import domolibrary.classes.DomoUser as dmu

auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)
# creates a DomoAccount object
domo_account = await DomoAccount.get_by_id(auth=auth, account_id=5)
domo_user = await dmu.DomoUser.get_by_id(user_id=68216396, auth=auth)

await domo_account.upsert_share_account_user(domo_user=domo_user, is_v2=True)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()